# Scraping CIA Data [Tutorial]
> "A quick tutorial on scraping and cleaing CIA factbook data"
- comments: true
- categories: [Tutorial, jupyter, Pandas, BeautifulSoup, spy]
- image: images/cia.jpg

We are interested in the CIA factbook data which is [located here](https://www.cia.gov/the-world-factbook/) (Please note this may have changed since 2020), I have downloaded the report from [here](https://www.cia.gov/library/publications/download/) and step by step we will read the data, clean it and presented in structured manner

In [28]:
import pandas as pd
import numpy as np
import statsmodels.api as sm 
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from IPython import display
import os
import re
import seaborn as sns; sns.set()
%matplotlib inline

# Scraping CIA Factbook

In [2]:
cat = 'CIA data/docs/notesanddefs.html'
page = open(cat).read()
page = BeautifulSoup(page)
data_map = {}
cols = page.select("div.category")
for col in cols:
    links = col.select('a')
    if len(links) > 0:
        fpath = links[0]['href']
        field = col.text.strip()
        data_map[field] =  fpath
        print(field, fpath)

Administrative divisions ../fields/302.html
Age structure ../fields/341.html
Agriculture - products ../fields/215.html
Airports ../fields/379.html
Airports - with paved runways ../fields/380.html
Airports - with unpaved runways ../fields/381.html
Area ../fields/279.html
Area - comparative ../fields/280.html
Background ../fields/325.html
Birth rate ../fields/345.html
Broadband - fixed subscriptions ../fields/206.html
Broadcast media ../fields/199.html
Budget ../fields/224.html
Budget surplus (+) or deficit (-) ../fields/226.html
Capital ../fields/301.html
Carbon dioxide emissions from consumption of energy ../fields/274.html
Central bank discount rate ../fields/230.html
Children under the age of 5 years underweight ../fields/368.html
Citizenship ../fields/310.html
Civil aircraft registration country code prefix ../fields/378.html
Climate ../fields/284.html
Coastline ../fields/282.html
Commercial bank prime lending rate ../fields/231.html
Communications - note ../fields/205.html
Constitu

So these are all the fields in the report, we will try now to map each field to its data while also keeping track of the country 

In [7]:
def map_data(data_map):
    data = {}
    for field in data_map:
        page = open('CIA data/fields/' + data_map[field].split('/')[-1]).read()
        page_field = BeautifulSoup(page)
        cols = page_field.select('td')
        for i in range(len(cols)):
            if i % 2 == 0:
                country = cols[i].select('a')[0].text
            else:
                value = cols[i].select('div.category_data')
                if field not in data:
                    data[field] = [(country, [x.text for x in value])]
                else:
                    data[field].append((country, [x.text for x in value]))
                    
    return data
data = map_data(data_map)       

# Demographics and Constructing a data Frame

We will start first with the demographics and answer general questions about each country, below are the columns of interest

In [472]:
Columns = ['Age structure', 'Area','Budget', 'Birth rate', 'Death rate', 'Debt - external', 'GDP (official exchange rate)',
          'GDP - per capita (PPP)', 'Hospital bed density','Physicians density', 'Life expectancy at birth',
           'Household income or consumption by percentage share', 'Population', 'Population below poverty line',
           'Religions', 'Unemployment rate', 'Median age']

In [473]:
from functools import reduce
dfList = []
for field in Columns:
    df = pd.DataFrame({'Country': [key[0] for key in data[field]], 
                       field: [ key[1] for key in data[field]]})
    dfList.append(df)
    

df = reduce(lambda x, y: pd.merge(x, y, how='outer', on = 'Country'), dfList)

In [474]:
df.head()

,Country,Age structure,Area,Budget,Birth rate,Death rate,Debt - external,GDP (official exchange rate),GDP - per capita (PPP),Hospital bed density,Physicians density,Life expectancy at birth,Household income or consumption by percentage share,Population,Population below poverty line,Religions,Unemployment rate,Median age
0,Afghanistan,"[\n0-14 years:\n40.92%\n(male 7,263,716 /femal...","[\ntotal:\n652,230 sq km\n\n, \nland:\n652,230...","[\nrevenues:\n2.276 billion\n\n(2017 est.)\n, ...","[\n37.5 births/1,000 population\n\n(2018 est.)\n]","[\n13.2 deaths/1,000 population\n\n(2018 est.)\n]",[\n$2.84 billion\n\n(FY/)\n],[\n$20.24 billion\n(2017 est.)\n(2017 est.)\n],"[\n$2,000\n\n(2017 est.)\n, \n$2,000\n\n(2016 ...","[\n0.5 beds/1,000 population\n\n(2014)\n]","[\n0.3 physicians/1,000 population\n\n(2016)\n]",[\ntotal population:\n52.1 years\n\n(2018 est....,"[\nlowest 10%:\n3.8%\n\n(2008)\n, \nhighest 10...","[\n34,940,837\n\n(July 2018 est.)\n]",[\n54.5%\n\n(2017 est.)\n],[\n \n Muslim 99.7% (Sunni 84....,"[\n23.9%\n\n(2017 est.)\n, \n22.6%\n\n(2016 es...","[\ntotal:\n19 years\n\n, \nmale:\n19 years\n\n..."
1,Albania,"[\n0-14 years:\n17.84%\n(male 287,750 /female ...","[\ntotal:\n28,748 sq km\n\n, \nland:\n27,398 s...","[\nrevenues:\n3.614 billion\n\n(2017 est.)\n, ...","[\n13.2 births/1,000 population\n\n(2018 est.)\n]","[\n6.9 deaths/1,000 population\n\n(2018 est.)\n]",[\n$9.505 billion\n\n(31 December 2017 est.)\n...,[\n$13.07 billion\n(2017 est.)\n(2017 est.)\n],"[\n$12,500\n\n(2017 est.)\n, \n$12,100\n\n(201...","[\n2.9 beds/1,000 population\n\n(2013)\n]","[\n1.29 physicians/1,000 population\n\n(2013)\n]",[\ntotal population:\n78.6 years\n\n(2018 est....,"[\nlowest 10%:\n19.6%\n\n(2015 est.)\n, \nhigh...","[\n3,057,220\n\n(July 2018 est.)\n]",[\n14.3%\n\n(2012 est.)\n],"[\n \n Muslim 56.7%, Roman Cat...","[\n13.8%\n\n(2017 est.)\n, \n15.2%\n\n(2016 es...","[\ntotal:\n33.4 years\n\n, \nmale:\n32 years\n..."
2,Algeria,"[\n0-14 years:\n29.49%\n(male 6,290,619 /femal...","[\ntotal:\n2,381,740 sq km\n\n, \nland:\n2,381...","[\nrevenues:\n54.15 billion\n\n(2017 est.)\n, ...","[\n21.5 births/1,000 population\n\n(2018 est.)\n]","[\n4.3 deaths/1,000 population\n\n(2018 est.)\n]","[\n$6.26 billion\n\n(31 December 2017 est.)\n,...",[\n$167.6 billion\n(2017 est.)\n(2017 est.)\n],"[\n$15,200\n\n(2017 est.)\n, \n$15,200\n\n(201...","[\n1.9 beds/1,000 population\n\n(2015)\n]",NaN,[\ntotal population:\n77.2 years\n\n(2018 est....,"[\nlowest 10%:\n26.8%\n\n(1995)\n, \nhighest 1...","[\n41,657,488\n\n(July 2018 est.)\n]",[\n23%\n\n(2006 est.)\n],[\n \n Muslim (official; predo...,"[\n11.7%\n\n(2017 est.)\n, \n10.5%\n\n(2016 es...","[\ntotal:\n28.3 years\n\n, \nmale:\n28 years\n..."
3,American Samoa,"[\n0-14 years:\n29.59%\n(male 7,732 /female 7,...","[\ntotal:\n224 sq km\n\n, \nland:\n224 sq km\n...","[\nrevenues:\n249 million\n\n(2016 est.)\n, \n...","[\n19 births/1,000 population\n\n(2018 est.)\n]","[\n5.9 deaths/1,000 population\n\n(2018 est.)\n]",[\nNA\n],[\n$658 million\n(2016 est.)\n(2016 est.)\n],"[\n$11,200\n\n(2016 est.)\n, \n$11,300\n\n(201...",NaN,NaN,[\ntotal population:\n73.9 years\n\n(2018 est....,"[\nlowest 10%:\nNA\n, \nhighest 10%:\nNA\n]","[\n50,826\n\n(July 2018 est.)\n]",[\nNA\n],"[\n \n Christian 98.3%, other ...",[\n29.8%\n\n(2005)\n],"[\ntotal:\n26.1 years\n\n, \nmale:\n25.6 years..."
4,Andorra,"[\n0-14 years:\n14.06%\n(male 6,197 /female 5,...","[\ntotal:\n468 sq km\n\n, \nland:\n468 sq km\n...","[\nrevenues:\n1.872 billion\n\n(2016)\n, \nexp...","[\n7.3 births/1,000 population\n\n(2018 est.)\n]","[\n7.4 deaths/1,000 population\n\n(2018 est.)\n]",[\n$0\n\n(2016)\n],[\n$2.712 billion\n(2016 est.)\n(2016 est.)\n],"[\n$49,900\n\n(2015 est.)\n, \n$51,300\n\n(201...","[\n2.5 beds/1,000 population\n\n(2009)\n]","[\n3.69 physicians/1,000 population\n\n(2015)\n]",[\ntotal population:\n82.9 years\n\n(2018 est....,"[\nlowest 10%:\nNA\n, \nhighest 10%:\nNA\n]","[\n85,708\n\n(July 2018 est.)\n]",NaN,[\n \n Roman Catholic (predom.

## Age Strucuture

as we can see, we managed to place data in a dataframe, however, it is not quite readable and contains descriptions and a lot of redunduncy that we can remove. We are dividing our data into 4 main tables:
   - Age structure
   - Median age
   - Demographics
   - Economics
    
We will alo rename our columns and structure our data such that its easy and ready for analysis

The `apply` method applies a transformation along an axis specified, in my opinion it's one of the most important methods in the pandas library and as you will see, we wil be using it almost in every block of code in order to clean or transform our data.

In [321]:
Age_groups = ['0-14', '15-24', '25-54', '55-64', '65+']
for i, age in enumerate(Age_groups):    
    df[age+'_male'] = df['Age structure'].apply(lambda l: re.search('male \d*,?\d*,?\d*', str(l[i])) 
                                                                    if isinstance(l, list) else float('Nan'))
    df[age+'_female'] = df['Age structure'].apply(lambda l: re.search('female \d*,?\d*,?\d*', str(l[i])) 
                                                                    if isinstance(l, list) else float('Nan'))
    
# cleaning up and turning into some readable numbers
def get_number(reg):
    if isinstance(reg, re.Match):
        return float(reg.string[reg.start(): reg.end()].split(' ')[-1].replace(',', ''))
    
    else:
        return float('nan')

In [322]:
age_columns = []
for i, age in enumerate(Age_groups):  
    df[age+'_male'] = df[age+'_male'].apply(get_number)
    df[age+'_female'] = df[age+'_female'].apply(get_number)
    df[age] = df[age+'_male'] + df[age+'_female']
    age_columns += [age+'_male', age+'_female']

In [323]:
Frame = pd.melt(df, id_vars=['Country'] , value_vars=age_columns,
        var_name='Category', value_name='Population')

In [324]:
Frame.head()

,Country,Category,Population
0,Afghanistan,0-14_male,7263716.0
1,Albania,0-14_male,287750.0
2,Algeria,0-14_male,6290619.0
3,American Samoa,0-14_male,7732.0
4,Andorra,0-14_male,6197.0


df

In [499]:
Frame[['Category', 'Sex']] = Frame['Category'].str.split('_', expand=True)
Frame.head()

,Country,Category,Population,Sex
0,Afghanistan,0-14,7263716.0,male
1,Albania,0-14,287750.0,male
2,Algeria,0-14,6290619.0,male
3,American Samoa,0-14,7732.0,male
4,Andorra,0-14,6197.0,male


In [297]:
len(df) == len(Frame)/5/2 # 5 age cateories and 2 sex cat

True

In [487]:
## median age:
median_df = df.copy()[['Country', 'Median age']]
median_df.head()

,Country,Median age
0,Afghanistan,"[\ntotal:\n19 years\n\n, \nmale:\n19 years\n\n..."
1,Albania,"[\ntotal:\n33.4 years\n\n, \nmale:\n32 years\n..."
2,Algeria,"[\ntotal:\n28.3 years\n\n, \nmale:\n28 years\n..."
3,American Samoa,"[\ntotal:\n26.1 years\n\n, \nmale:\n25.6 years..."
4,Andorra,"[\ntotal:\n44.9 years\n\n, \nmale:\n45.1 years..."


In [491]:
median_df['median_age'] = median_df['Median age'].apply(lambda t: float(t[0].split('\n')[2].split()[0])
                                                    if isinstance(t, list) else float('nan'))

median_df['median_male'] = median_df['Median age'].apply(lambda t: float(t[1].split('\n')[2].split()[0])
                                                    if isinstance(t, list) else float('nan'))

median_df['median_female'] = median_df['Median age'].apply(lambda t: float(t[2].split('\n')[2].split()[0])
                                                    if isinstance(t, list) else float('nan'))

In [495]:
median_df.drop('Median age', axis=1, inplace=True)

In [496]:
median_df.head()

,Country,median,median_male,median_female
0,Afghanistan,19.0,19.0,19.1
1,Albania,33.4,32.0,34.7
2,Algeria,28.3,28.0,28.7
3,American Samoa,26.1,25.6,26.5
4,Andorra,44.9,45.1,44.8


## More Demographics by Country

In [298]:
demo = ['Area','Birth rate', 'Death rate', 'Hospital bed density','Physicians density', 
        'Life expectancy at birth','Population below poverty line','Unemployment rate']

In [299]:
for c in demo:
    df[c] = df[c].apply(lambda l: l[0].replace('\n', '') if isinstance(l , list) else float('nan'))

In [300]:
demo_df = df.copy()[demo]
demo_df.head()

,Area,Birth rate,Death rate,Hospital bed density,Physicians density,Life expectancy at birth,Population below poverty line,Unemployment rate
0,"total:652,230 sq km","37.5 births/1,000 population(2018 est.)","13.2 deaths/1,000 population(2018 est.)","0.5 beds/1,000 population(2014)","0.3 physicians/1,000 population(2016)",total population:52.1 years(2018 est.),54.5%(2017 est.),23.9%(2017 est.)
1,"total:28,748 sq km","13.2 births/1,000 population(2018 est.)","6.9 deaths/1,000 population(2018 est.)","2.9 beds/1,000 population(2013)","1.29 physicians/1,000 population(2013)",total population:78.6 years(2018 est.),14.3%(2012 est.),13.8%(2017 est.)
2,"total:2,381,740 sq km","21.5 births/1,000 population(2018 est.)","4.3 deaths/1,000 population(2018 est.)","1.9 beds/1,000 population(2015)",NaN,total population:77.2 years(2018 est.),23%(2006 est.),11.7%(2017 est.)
3,total:224 sq km,"19 births/1,000 population(2018 est.)","5.9 deaths/1,000 population(2018 est.)",NaN,NaN,total population:73.9 years(2018 est.),NA,29.8%(2005)
4,total:468 sq km,"7.3 births/1,000 population(2018 est.)","7.4 deaths/1,000 population(2018 est.)","2.5 beds/1,000 population(2009)","3.69 physicians/1,000 population(2015)",total population:82.9 years(2018 est.),NaN,3.7%(2016 est.)


### Area (km^2) & Life expectancy at birth

In [301]:
area = demo_df['Area'].apply(lambda x: (x.split(':')[1].split(' ')[0]).replace(',',''))
area.at[243] = 439781 #French Southern and Antarctic Lands'
demo_df['Area'] = area.astype(float)

In [302]:
def get_exp(c):
    try:
        return float((c.split(':')[1].split(' ')[0]).replace(',',''))
    except:
        return float('nan')
        
life_exp = df['Life expectancy at birth'].apply(get_exp)
demo_df['Life expectancy at birth'] = life_exp.astype(float)

In [303]:
demo_df

,Area,Birth rate,Death rate,Hospital bed density,Physicians density,Life expectancy at birth,Population below poverty line,Unemployment rate
0,652230.00,"37.5 births/1,000 population(2018 est.)","13.2 deaths/1,000 population(2018 est.)","0.5 beds/1,000 population(2014)","0.3 physicians/1,000 population(2016)",52.1,54.5%(2017 est.),23.9%(2017 est.)
1,28748.00,"13.2 births/1,000 population(2018 est.)","6.9 deaths/1,000 population(2018 est.)","2.9 beds/1,000 population(2013)","1.29 physicians/1,000 population(2013)",78.6,14.3%(2012 est.),13.8%(2017 est.)
2,2381740.00,"21.5 births/1,000 population(2018 est.)","4.3 deaths/1,000 population(2018 est.)","1.9 beds/1,000 population(2015)",NaN,77.2,23%(2006 est.),11.7%(2017 est.)
3,224.00,"19 births/1,000 population(2018 est.)","5.9 deaths/1,000 population(2018 est.)",NaN,NaN,73.9,NA,29.8%(2005)
4,468.00,"7.3 births/1,000 population(2018 est.)","7.4 deaths/1,000 population(2018 est.)","2.5 beds/1,000 population(2009)","3.69 physicians/1,000 population(2015)",82.9,NaN,3.7%(2016 est.)
...,...,...,...,...,...,...,...,...
262,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,62045.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264,12.00,NaN,NaN,NaN,"2.72 physicians/1,000 population(2010)",NaN,NA,NA
265,6959.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Birth Rate, Death rate, Physicians density, Hospital bed density, Population below poverty line and Unemployment rate

In [304]:
section1 = ['Birth rate', 'Death rate', 'Hospital bed density','Physicians density']

In [305]:
for s in section1:
    demo_df[s] = demo_df[s].map(lambda x: float(x.split(' ')[0]) if isinstance(x, str) else float('nan'))

In [306]:
section2 = ['Population below poverty line', 'Unemployment rate']
def clean_perc(c):
    try:
        return float(c.split('%')[0])
    except:
        return float('nan')
        
for s in section2:
    demo_df[s] = demo_df[s].apply(clean_perc)

In [312]:
demo_df.index = df['Country']

## GDP and Budget by Country

In [452]:
selection = ['Country', 'Budget', 'Debt - external', 'GDP (official exchange rate)', 'GDP - per capita (PPP)']
eco_df = df.copy()[selection]

,Country,Budget,Debt - external,GDP (official exchange rate),GDP - per capita (PPP)
0,Afghanistan,"[\nrevenues:\n2.276 billion\n\n(2017 est.)\n, ...",[\n$2.84 billion\n\n(FY/)\n],[\n$20.24 billion\n(2017 est.)\n(2017 est.)\n],"[\n$2,000\n\n(2017 est.)\n, \n$2,000\n\n(2016 ..."
1,Albania,"[\nrevenues:\n3.614 billion\n\n(2017 est.)\n, ...",[\n$9.505 billion\n\n(31 December 2017 est.)\n...,[\n$13.07 billion\n(2017 est.)\n(2017 est.)\n],"[\n$12,500\n\n(2017 est.)\n, \n$12,100\n\n(201..."
2,Algeria,"[\nrevenues:\n54.15 billion\n\n(2017 est.)\n, ...","[\n$6.26 billion\n\n(31 December 2017 est.)\n,...",[\n$167.6 billion\n(2017 est.)\n(2017 est.)\n],"[\n$15,200\n\n(2017 est.)\n, \n$15,200\n\n(201..."
3,American Samoa,"[\nrevenues:\n249 million\n\n(2016 est.)\n, \n...",[\nNA\n],[\n$658 million\n(2016 est.)\n(2016 est.)\n],"[\n$11,200\n\n(2016 est.)\n, \n$11,300\n\n(201..."
4,Andorra,"[\nrevenues:\n1.872 billion\n\n(2016)\n, \nexp...",[\n$0\n\n(2016)\n],[\n$2.712 billion\n(2016 est.)\n(2016 est.)\n],"[\n$49,900\n\n(2015 est.)\n, \n$51,300\n\n(201..."


In [453]:
#Budget
def extract_budget(l, i):
    try:
        return l[i].split('\n')[2]
    except:
        return float('nan')
    
eco_df['Revenues($)'] = eco_df['Budget'].map(lambda t: extract_budget(t, 0))
eco_df['Expenditures($)'] = eco_df['Budget'].map(lambda t: extract_budget(t, 1))

In [454]:
eco_df['Debt_ext($)'] = eco_df['Debt - external'].map(lambda t: t[0].split('\n')[1] 
                                                   if isinstance(t, list) else float('nan'))
eco_df['GDP($)'] = eco_df['GDP (official exchange rate)'].map(lambda t: t[0].split('\n')[1] 
                                                   if isinstance(t, list) else float('nan'))
eco_df['GDP_per_Capita($)'] = eco_df['GDP - per capita (PPP)'].map(lambda t: t[0].split('\n')[1] 
                                                   if isinstance(t, list) else float('nan'))

In [455]:
eco_df.drop(['Budget', 'Debt - external', 'GDP (official exchange rate)', 'GDP - per capita (PPP)']
            , axis=1, inplace=True)

In [456]:
eco_df.head()

,Country,Revenues($),Expenditures($),Debt_ext($),GDP($),GDP_per_Capita($)
0,Afghanistan,2.276 billion,5.328 billion,$2.84 billion,$20.24 billion,"$2,000"
1,Albania,3.614 billion,3.874 billion,$9.505 billion,$13.07 billion,"$12,500"
2,Algeria,54.15 billion,70.2 billion,$6.26 billion,$167.6 billion,"$15,200"
3,American Samoa,249 million,262.5 million,NA,$658 million,"$11,200"
4,Andorra,1.872 billion,2.06 billion,$0,$2.712 billion,"$49,900"


In [457]:
def clean_fig(e, mode=1):
    try:
        l = e.split(' ')
        if mode == 1:
            num = float(l[0].replace(',', ''))
        else:
            num = float(l[0][1:].replace(',', ''))
        if len(l) == 1:
            return num
        else:
            return num * 1e6 if l[1].strip() == 'million' else num * 1e9
    except:
        return float('nan')

In [458]:
eco_df['Revenues($)'] = eco_df['Revenues($)'].apply(clean_fig)
eco_df['Expenditures($)'] = eco_df['Expenditures($)'].apply(clean_fig)
eco_df['Debt_ext($)'] = eco_df['Debt_ext($)'].apply(lambda t: clean_fig(t, 2))
eco_df['GDP($)'] = eco_df['GDP($)'].apply(lambda t: clean_fig(t, 2))
eco_df['GDP_per_Capita($)'] = eco_df['GDP_per_Capita($)'].apply(lambda t: clean_fig(t, 2))

In [459]:
eco_df.head(5)

,Country,Revenues($),Expenditures($),Debt_ext($),GDP($),GDP_per_Capita($)
0,Afghanistan,2.276000e+09,5.328000e+09,2.840000e+09,2.024000e+10,2000.0
1,Albania,3.614000e+09,3.874000e+09,9.505000e+09,1.307000e+10,12500.0
2,Algeria,5.415000e+10,7.020000e+10,6.260000e+09,1.676000e+11,15200.0
3,American Samoa,2.490000e+08,2.625000e+08,NaN,6.580000e+08,11200.0
4,Andorra,1.872000e+09,2.060000e+09,0.000000e+00,2.712000e+09,49900.0


## Overview so far

In [500]:
Frame.sample(n=10)

,Country,Category,Population,Sex
955,Norway,15-24,324088.0,female
1867,United States Pacific Island Wildlife Refuges,55-64,NaN,male
945,Namibia,15-24,257984.0,female
1028,World,15-24,572229547.0,female
492,West Bank,0-14,491676.0,female
577,Christmas Island,15-24,202.0,male
2475,France,65+,7569011.0,female
1373,Cayman Islands,25-54,12855.0,female
421,Norway,0-14,471014.0,female
2204,European Union,65+,43673572.0,male


In [501]:
Frame.to_csv('clean_CIA_data/Age_structure.csv')

In [497]:
median_df.sample(n=10)

,Country,median,median_male,median_female
201,Tajikistan,24.8,24.2,25.4
23,Bhutan,28.1,28.6,27.6
207,Trinidad and Tobago,36.6,36.1,37.1
184,Sierra Leone,19.1,18.4,19.7
213,Uganda,15.9,15.8,16.0
70,Fiji,29.2,29.0,29.4
101,Israel,30.1,29.5,30.7
144,Namibia,21.4,20.7,22.2
10,Aruba,39.5,37.8,41.2
3,American Samoa,26.1,25.6,26.5


In [498]:
median_df.to_csv('clean_CIA_data/median_age_country.csv')

In [461]:
demo_df = demo_df.rename(columns={'Area': 'Area(km^2)',
                        'Birth rate': 'Birth_rate_per_1000',
                        'Death rate': 'Birth_rate_per_1000',
                        'Hospital bed density': 'Hospital_bed_density_per_1000',
                        'Life expectancy at birth': 'Life_expec_at_birth',
                        'Physicians density': 'Physicians_density_per_1000',
                        'Population below poverty line': 'Percentage_below_poverty_line',
                        'Unemployment rate': 'Unemployment_rate_%'})

In [462]:
demo_df.sample(n=10)

,Area(km^2),Birth_rate_per_1000,Birth_rate_per_1000,Hospital_bed_density_per_1000,Physicians_density_per_1000,Life_expec_at_birth,Percentage_below_poverty_line,Unemployment_rate_%
Country,,,,,,,,
South Sudan,644329.0,36.9,19.3,NaN,NaN,NaN,66.0,NaN
Western Sahara,266000.0,28.9,7.9,NaN,NaN,63.8,NaN,NaN
Saint Vincent and the Grenadines,389.0,13.0,7.4,2.6,NaN,75.8,NaN,18.8
Panama,75420.0,17.6,5.0,2.3,1.59,78.9,23.0,6.0
Malta,316.0,10.0,7.9,4.7,3.91,82.7,16.3,4.6
Wallis and Futuna,142.0,13.0,5.5,NaN,1.10,80.0,NaN,8.8
Denmark,43094.0,10.9,9.3,2.5,3.66,81.0,13.4,5.7
Kiribati,811.0,21.0,7.0,1.9,0.20,66.9,NaN,30.6
Greece,131957.0,8.3,11.4,4.3,6.26,80.8,36.0,21.5


In [469]:
demo_df.to_csv('clean_CIA_data/country_data.csv')

In [466]:
eco_df.index = eco_df['Country']; eco_df.drop('Country', axis=1, inplace=True)
eco_df.sample(n=10)

,Revenues($),Expenditures($),Debt_ext($),GDP($),GDP_per_Capita($)
Country,,,,,
Uganda,3.848000e+09,4.928000e+09,1.080000e+10,2.662000e+10,2400.0
Jan Mayen,NaN,NaN,NaN,NaN,NaN
Belize,5.535000e+08,5.720000e+08,1.315000e+09,1.854000e+09,8300.0
Bahrain,5.854000e+09,9.407000e+09,5.215000e+10,3.533000e+10,49000.0
Honduras,4.658000e+09,5.283000e+09,8.625000e+09,2.298000e+10,5600.0
Kyrgyzstan,2.169000e+09,2.409000e+09,8.164000e+09,7.565000e+09,3700.0
Jordan,9.462000e+09,1.151000e+10,2.934000e+10,4.013000e+10,9200.0
Mongolia,2.967000e+09,3.681000e+09,2.533000e+10,1.114000e+10,13000.0
Sweden,2.712000e+11,2.644000e+11,9.399000e+11,5.356000e+11,51200.0


In [471]:
eco_df.to_csv('clean_CIA_data/eco_overview.csv')

# Conclusion
Now you are ready to analyze the world factbook data. We can have went over a bunch of techniques and libraries to help us scrape a web page:
- BeautifulSoup: parse HTML content and get fields we need
- pandas: structure the data and apply manipulation
- re: regular expression library to helps us clean data

in addition we can use `requests` to import the HTML content.

Hopefully this guided code provided you with an idea on how you can apply the same to your own.